<a href="https://colab.research.google.com/github/armando-lv/CSVsDatosDenue/blob/main/Denue_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook sirve para consultar el API del DENUE realizado por el INEGI.
Realiza una consulta BuscarEntidad, guardando en tu Drive 32 CSVs con información de todos los negocios pertenecientes a cada estado.

En la sección *División de CSVs* se pueden dividir estos archivos para que tengan un menor tamaño. Estos se guardan en un folder aparte.

Es necesario tener una clave de acceso para usar el notebook, la cual la puedes conseguir aquí: https://www.inegi.org.mx/servicios/api_denue.html


#### Consulta y descarga de datos del API

In [ ]:
# Importamos las librerías que usaremos
import requests
import pandas as pd

In [ ]:
# Montamos el google drive donde queremos nuestros CSVs, recuerda seguir las instrucciones
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Inserta como string tu clave de acceso
key = "[TU CLAVE AQUI]"

In [ ]:
# Los parametros que usaremos para nuestra consulta, en drive_folder debes poner el folder de tu google drive donde deseas guardar tus archivos
cond = "todos"
rinicial = 1
rfinal = 1000000
drive_folder = "/content/drive/[Inserta el resto de tu directorio aqui]"

In [ ]:
# Los estados que consultaremos, y un valor de relleno para hacer más facil los for loops
states = [
    "Filler", "Aguascalientes", "Baja California", "Baja California Sur", "Campeche", "Coahuila", "Colima", "Chiapas", 
    "Chihuahua", "Ciudad de México", "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "Estado de México", 
    "Michoacán", "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", "Querétaro", "Quintana Roo", "San Luis Potosí", 
    "Sinaloa", "Sonora", "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatán", "Zacatecas"
]

In [ ]:
for i in range(1, 33):
  # Si queremos consultar Aguascalientes, el valor debe ser "01" y no solo "1"
  if len(str(i)) == 1:
    estado = "0" + str(i)
  else:
    estado = str(i)

  # Obtenemos nuestros datos de la API
  bus_endp = f"https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarEntidad/{cond}/{estado}/{rinicial}/{rfinal}/{key}"
  bus = requests.get(bus_endp)
  data = bus.json()
  df = pd.json_normalize(data)

  # Guardamos el resultado en un csv, dentro del folder que insertaste y bajo el nombre del estado
  df.to_csv(f"{drive_folder}{states[int(estado)]}.csv", index=False)

  # Todo esto aplica para cada estado, resultando en un folder con 32 csvs

In [ ]:
# Estos servirán para crear un dashboard ;)
df_c = df[["Nombre", "Clase_actividad", "Estrato", "Ubicacion", "Longitud", "Latitud"]]
df_c[["Localidad", "Municipio", "Estado"]] = df_a["Ubicacion"].str.split(",", expand=True)
df_c.drop(columns="Ubicacion")

#### División de CSVs

In [ ]:
import os
import math
import numpy as np
# Github sólo permite archivos de máximo 25 mb, así que algunos de estos csvs tendran que ser divididos

In [ ]:
# De antemano creamos un folder para guardar nuestros sets divididos
ext = "[Nombre del Folder]/"

In [ ]:
for i in range(1, 33):
  filesize = round(os.path.getsize(f"{drive_folder}{states[i]}.csv") / 1048576, 2)
  if filesize < 20:
    continue
  parts = math.ceil(filesize / 20)
  csv = pd.read_csv(f"{drive_folder}{states[i]}.csv")
  parts_array = np.array_split(csv, parts)
  for x in range (1, parts+1):
    parts_array[x-1].to_csv(f"{drive_folder}{ext}{states[i]}-{x}.csv", index=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
